In [4]:
!pwd
!mkdir -p /content
%cd /content

/app
/content


In [9]:
!apt install python-is-python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  python-is-python3
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 2788 B of archives.
After this operation, 13.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 python-is-python3 all 3.9.2-2 [2788 B]
Fetched 2788 B in 1s (4663 B/s)             
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package python-is-python3.
(Reading database ... 44398 files and directories currently installed.)
Preparing to unpack .../python-is-python3_3.9.2-2_all.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking python-is-python3 (3.9.2-2) ...
7Progress: [ 40%] [#######################...........

In [6]:
!echo y | pip3 uninstall transformers
!echo y | pip3 uninstall optimum
!echo y | pip3 uninstall auto-gptq

#!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/optimum.git
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Found existing installation: transformers 4.32.1
Uninstalling transformers-4.32.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.32.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)?   Successfully uninstalled transformers-4.32.1
Found existing installation: optimum 1.12.0
Uninstalling optimum-1.12.0:
  Would remove:
    /usr/local/bin/optimum-cli
    /usr/local/lib/python3.10/dist-packages/optimum-1.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/*
    /usr/local/lib/python3.10/dist-packages/optimum/commands/*
    /usr/local/lib/python3.10/dist-packages/optimum/configuration_utils.py
    /usr/local/lib/python3.10/dist-packages/optimum/conftest.py
    /usr/local/lib/python3.10/dist-packages/optimum/exporters/*
    /usr/local/lib/python3.10/dist-packages/optimum/fx/*
    /usr/local/lib/python3.10/dist-packages/optimum/gptq/*
    /usr/local/lib/pyth

In [ ]:
!pip3 install --upgrade accelerate
# !echo y | pip3 uninstall bitsandbytes

In [1]:
# 最初の一度だけ実行
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Yarn-Llama-2-13B-64K-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             # load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main",
                                            )

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

>>>> Flash Attention installed
>>>> Flash RoPE installed


In [3]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./llama2論文.pdf")
documents = loader.load()

print(len(documents))
document=""
for doc in documents:
    document+=doc.page_content
text=document.replace("\n","")
print(len(text))

text = text[:5000]

77
248584
Result: 次の学術論文を読みます。

 Llama 2 : Open Foundation and Fine-Tuned Chat ModelsHugo Touvron∗Louis Martin†Kevin Stone†Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya BatraPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya ChenGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian FullerCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui HouHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem KorenevPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana LiskovichYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar MishraIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan SaladiAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh TangRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen ZhangAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert S

In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./nextjs-vs-remixjs.txt")
documents = loader.load()

print(len(documents))
document=""
for doc in documents:
    document+=doc.page_content
text=document.replace("\n","")
print(len(text))

text = text[:10000]

del pipe
torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

question="Which is better, Next.js or Remix.js?"
sequences = pipe(
    f"Read the following article and answer the Questions.\n\n{text}\n\n{question}\n",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=5000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

1
39189


In [24]:
del pipe
torch.cuda.empty_cache()

from langchain.document_loaders import TextLoader

loader = TextLoader("./mushoku.txt")
documents = loader.load()

document=""
for doc in documents:
    document+=doc.page_content
text=document.replace("\n","")
print(f"Document length: {len(text)}")

text = text[:10000]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

question="The dead are listed below."
sequences = pipe(
    f"Read the following article and answer the Questions.\n\n{text}\n\n{question}\n",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=5000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
    
question="The names of the persons are listed below."
sequences = pipe(
    f"Read the following article and answer the Questions.\n\n{text}\n\n{question}\n",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=5000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
    
    
question="I taught Rudeus magic,"
sequences = pipe(
    f"Read the following article and answer the Questions.\n\n{text}\n\n{question}\n",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=5000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Document length: 2347
Result: Read the following article and answer the Questions.

Synopsis of Unemployed ReincarnationThe main character, 34 years old, unemployed, NEET and living a reclusive life, is run over by a truck one day and dies.He finds himself reincarnated as a baby in a different world reminiscent of medieval Europe.His name is Ludeus Greylat, and he is determined to live a good life the next time.From a young age, he read many books and practiced magic.His parents saw that he had talent and brought in Roxy Migurdia as a magical tutor.Later, after having two younger sisters and completing his magical training, Rudaeus began to worry about his own development as he grew older.He asked his father, Paulo, if he and his friend Sylphiette, a beautiful girl, could attend a magic college, but his father would not allow it and sent them to the fortress city of Roa.Rudeus is sent to the Citadel of Roa to work as a tutor for the young but violent Elise Boreas Greylat in order to ea

In [7]:
del pipe
torch.cuda.empty_cache()

from langchain.document_loaders import TextLoader

loader = TextLoader("./bocchi.txt")
documents = loader.load()

document=""
for doc in documents:
    document+=doc.page_content
text=document.replace("\n","")
print(f"Document length: {len(text)}")

# text = text[:20000]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

question="User:What is the name of the manager of the live house \"STARRY\"?"
sequences = pipe(
    f"Read the following article and answer the Questions.\n\n{text}\n\n{question}\nAssistant:",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=5000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Document length: 47361


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.51 GiB (GPU 0; 23.67 GiB total capacity; 17.66 GiB already allocated; 1.42 GiB free; 20.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF